In [ ]:
import mysql.connector
import sys
sys.path.append('..')
from config import config
from pathlib import Path
import os
import json
import re
import datetime
import logging
import timeit
import csv
import matplotlib.pyplot as plt
from sentiment import sentiment_score

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
# import time

from transformers import logging as transformers_logging
transformers_logging.set_verbosity_error()

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
configr = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [ ]:
label, score = sentiment_score.roberta("I work 40 hours a week for me to be this poor", tokenizer, model, configr)
print(label)
print(score)

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()
analyzer.polarity_scores("I work 40 hours a week for me to be this poor")

In [ ]:
connection = mysql.connector.connect(host=config.get('HOST'), user=config.get('USERNAME'), password=config.get('PASSWORD'),database=config.get('DATABASE'), allow_local_infile=True)

In [ ]:
cursor = connection.cursor()
cursor.execute("SELECT * FROM tweets ORDER BY timestamp_ms DESC")
tweets = cursor.fetchall()

In [ ]:
def add_sentiment_column(connection):
    add_column_query = """
    ALTER TABLE `tweets`
    ADD COLUMN `sentiment` FLOAT NULL;
    """
    connection.cursor().execute(add_column_query)

In [ ]:
add_sentiment_column(connection)

In [ ]:
def process_tweet(tweet):
    id, text = tweet
    try:
        score = sentiment_score.roberta(text)
        return (id, score)
    except Exception as e:
        print(f"❌ Error: {e}, Tweet: {id}", file=sys.stderr)
        return None

In [ ]:
import os
import timeit
import datetime

os.environ["TOKENIZERS_PARALLELISM"] = "false"

from multiprocessing import Pool

def update_sentiment_scores(connection):
    cursor = connection.cursor()
    cursor.execute("SELECT `id`, `text` FROM `tweets`")
    tweets = cursor.fetchall()
    total_tweets = len(tweets)

    counter = 0
    elapsed = 0
    errors = 0

    with Pool() as p:
        results = []
        for tweet in tweets:
            start = timeit.default_timer()
            res = process_tweet(tweet)
            results.append(res)
            duration = timeit.default_timer() - start
            elapsed += duration
            if counter == 0:
                time_remaining = 0  # or some default value
            else:
                time_remaining = (total_tweets - counter) * (elapsed / counter)
            print(f"Processed {counter} out of {total_tweets} tweets. Estimated time left: {str(datetime.timedelta(seconds=time_remaining))}")
            counter += 1

    # Filter out None results
    results = [r for r in results if r is not None]

    # Batch update
    update_query = "UPDATE `tweets` SET `sentiment` = CASE `id` "
    for id, score in results:
        update_query += f"WHEN {id} THEN {score} "
    update_query += "END WHERE `id` IN (" + ",".join(str(id) for id, _ in results) + ")"
    cursor.execute(update_query)
    connection.commit()
    print("Database update completed")

update_sentiment_scores(connection)

In [ ]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

def update_sentiment_scores_spark():
    # Create a SparkSession with the MySQL JDBC driver
    spark = SparkSession.builder \
        .config("spark.jars", "/path/to/mysql-connector-java-version-bin.jar") \
        .getOrCreate()

    # Read environment variables
    host = os.getenv('HOST')
    username = os.getenv('USERNAME')
    password = os.getenv('PASSWORD')
    database = os.getenv('DATABASE')

    # JDBC connection string
    url = f"jdbc:mysql://{host}:3306/{database}?user={username}&password={password}"

    # Load the tweets into a DataFrame
    df = spark.read.jdbc(url=url, table="tweets")

    # Define a UDF to calculate the sentiment score
    sentiment_score_udf = udf(lambda text: sentiment_score.roberta(text), FloatType())

    # Calculate the sentiment scores
    df = df.withColumn("sentiment", sentiment_score_udf(df["text"]))

    # Write the updated DataFrame back to the database
    df.write.jdbc(url=url, table="tweets", mode="overwrite")

update_sentiment_scores_spark()

In [ ]:
def update_sentiment_scores(connection):
    cursor = connection.cursor()
    cursor.execute("SELECT `id`, `text` FROM `tweets`")
    tweets = cursor.fetchall()

    counter = 0
    elapsed = 0
    n = len(tweets)
    errors = 0

    for tweet in tweets:
        start = timeit.default_timer()
        id, text = tweet
        try:
            score = sentiment_score.roberta(text)
            print(f"📊 Sentiment score: {score}")
            cursor.execute(f"UPDATE `tweets` SET `sentiment` = {score} WHERE `id` = {id}")
            if counter % round(n/10) == 0:
                connection.commit()
            print(f"✅ Tweet {id} updated.")
        except Exception as e:
            print(f"❌ Error: {e}, Tweet: {id}", file=sys.stderr)
            errors += 1

        counter += 1
        duration = timeit.default_timer() - start
        elapsed += duration
        time_remaining = (n - counter) * (elapsed / counter)
        print(f"⏯️ Process: {(counter/n)*100:.2f}% - #️⃣ {counter}/{n} tweets updated - ⏳ Time remaining : {str(datetime.timedelta(seconds=time_remaining))}")
        print("-----------------------------------")

update_sentiment_scores(connection)

In [ ]:
from transformers import pipeline
sentiment_task = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest", tokenizer="cardiffnlp/twitter-roberta-base-sentiment-latest")

In [ ]:
text = "I have an iphone"

In [ ]:
sentiment_task(text)